# Saving frames in parallel

In [1]:
import xarray as xr
from xmovie import Movie

ds = xr.tutorial.open_dataset('air_temperature').isel(time=slice(0,200))

In [2]:
# Creating the movie object
mov = Movie(ds.air, vmin=230, vmax=310)

The creation of a movie can take quite long for datasets with many timesteps, creating many frames in a loop.

In [3]:
%%time
mov.save('movie.mov', overwrite_existing=True)

Movie created at movie.mov
CPU times: user 34.7 s, sys: 1.02 s, total: 35.7 s
Wall time: 59.4 s


You can speed up the frame creation by activating the `parallel` option. This will save the frames using dask.

For this to work you need to chunk the input dataarray with a single step along the dimension that represent your frames (`framedim`).

In [5]:
mov_parallel = Movie(ds.air.chunk({'time':1}), vmin=230, vmax=310)

In [6]:
%%time
mov_parallel.save(
    'movie_parallel.mov',
    parallel=True,
    overwrite_existing=True,
)

Movie created at movie_parallel.mov
CPU times: user 38.8 s, sys: 1.46 s, total: 40.3 s
Wall time: 48.3 s


You can pass arguments to the dask `.compute()` call with `parallel_compute_kwargs` to tune for your particular setup.

In [7]:
%%time
mov_parallel.save(
    'movie_parallel_modified.mov',
    parallel=True,
    overwrite_existing=True,
    parallel_compute_kwargs=dict(scheduler="processes", num_workers=8)
)

Movie created at movie_parallel.mov
CPU times: user 4.84 s, sys: 249 ms, total: 5.09 s
Wall time: 33.6 s


Thats not bad, a 50% time saving (keeping in mind that the time needed for the ffmpeg call is included). We expect speedups to be even bigger when using higher resolution data.